<div style="text-align: center; padding: 20px;">
    <img src="austin-distel-744oGeqpxPQ-unsplash.jpeg" alt="​​Subscription Renewal Insights for a SaaS Company" width="450"/>
</div>


<!-- Image source - https://unsplash.com/photos/person-using-macbook-pro-744oGeqpxPQ
-->

## 📖 The Background
A SaaS company seeks to uncover what drives its clients to renew subscriptions. They’ve collected data on client details, subscription records, and economic indicators and would like to connect them to better understand its clients’ behavior.

They’ve tasked you with analyzing these datasets to identify the key factors influencing clients’ decisions to renew their subscriptions.  
Your analysis will provide them with insights into which customers are renewing their products and the reasons behind their renewals. The company can leverage these insights to make informed decisions to increase renewal rates and improve customer loyalty, helping them stay competitive and ensure long-term growth.


## 💾 The Data
The company have provided you with three datasets for your analysis. A summary of each data is provided below.

#### `client_details.csv`

| Column         | Description|
|----------------|---------------------------------------------------------------|
| `client_id`    | Unique identifier for each client. |
| `company_size` | Size of the company (Small, Medium, Large).|
| `industry`     | Industry to which the client belongs (Fintech, Gaming, Crypto, AI, E-commerce).|
| `location`     | Location of the client (New York, New Jersey, Pennsylvania, Massachusetts, Connecticut).|

#### `subscription_records.csv`

| Column             | Description   |
|--------------------|---------------|
| `client_id`        | Unique identifier for each client.|
| `subscription_type`| Type of subscription (Yearly, Monthly).|
| `start_date`       | Start date of the subscription - YYYY-MM-DD.|
| `end_date`         | End date of the subscription - YYYY-MM-DD.|
| `renewed`          | Indicates whether the subscription was renewed (True, False).|

#### `economic_indicators.csv`

| Column           | Description                                       |
|------------------|---------------------------------------------------|
| `start_date`     | Start date of the economic indicator (Quarterly) - YYYY-MM-DD.|
| `end_date`       | End date of the economic indicator (Quarterly) - YYYY-MM-DD.|
| `inflation_rate` | Inflation rate in the period.|
| `gdp_growth_rate`| Gross Domestic Product (GDP) growth rate in the period.|

# Objectives
1) How many total Fintech and Crypto clients does the company have?
2) Which industry has the highest renewal rate?
3) For clients that renewed their subscriptions, what was the average inflation rate when their subscriptions were renewed?

In [2]:
import pandas as pd
import numpy as np

In [4]:
#import data
client_df = pd.read_csv('client_details.csv')
subscrp_df = pd.read_csv('subscription_records.csv', parse_dates = ['start_date','end_date'])
eco_df = pd.read_csv('economic_indicators.csv', parse_dates = ['start_date','end_date']).rename(
    columns={'Unnamed: 0':'eco_indicator'})

#inspect data
display(client_df.head(3), subscrp_df.head(3), eco_df.head(3))
display(client_df.info(), subscrp_df.info(), eco_df.info())

,client_id,company_size,industry,location
0,4280387012,Large,Fintech,New York
1,2095513148,Small,Fintech,New Jersey
2,7225516707,Medium,Fintech,Pennsylvania


,client_id,subscription_type,start_date,end_date,renewed
0,1131383004,Yearly,2020-11-11,2021-11-11,False
1,4309371709,Monthly,2021-05-24,2021-06-23,True
2,3183675157,Yearly,2021-12-25,2022-12-25,True


,eco_indicator,start_date,end_date,inflation_rate,gdp_growth_rate
0,0,2018-01-01,2018-03-31,5.77,3.51
1,1,2018-04-01,2018-06-30,1.17,2.15
2,2,2018-07-01,2018-09-30,1.56,1.82


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   client_id     100 non-null    int64 
 1   company_size  100 non-null    object
 2   industry      100 non-null    object
 3   location      100 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   client_id          100 non-null    int64         
 1   subscription_type  100 non-null    object        
 2   start_date         100 non-null    datetime64[ns]
 3   end_date           100 non-null    datetime64[ns]
 4   renewed            100 non-null    bool          
dtypes: bool(1), datetime64[ns](2), int64(1), object(1)
memory usage: 3.4+ KB
<class 'pandas.core.frame.Dat

None

None

None

# Analysis I
**How many total Fintech and Crypto clients does the company have?**

In [10]:
#calculate # clients per industry
num_ind = client_df.groupby('industry')['client_id'].count().reset_index().rename(columns={'client_id':'# clients'})
#obtain # Fintech clients
num_Fintech = num_ind[num_ind.industry == 'Fintech']['# clients'].values[0]
#obtain # Crypto clients
num_Crypto = num_ind[num_ind.industry == 'Crypto']['# clients'].values[0]
#take sum
total_fintech_crypto_clients = num_Fintech + num_Crypto

display(f"The company has {num_Fintech} Fintech & {num_Crypto} Crypto clients ({total_fintech_crypto_clients} total).")

'The company has 22 Fintech & 25 Crypto clients (47 total).'

# Analysis II
**Which industry has the highest renewal rate?**

In [56]:
#need to merge client & subscription data, can use the 'client_id' variable
client_subscrp_df = client_df.merge(subscrp_df, on='client_id')

#obtain the # clients who renewed per industry & the # total clients per industry
ind_ren_nums = client_subscrp_df.groupby('industry').agg({'renewed':'sum', 'client_id':'count'}).reset_index().rename(
    columns={'renewed':'# renewed', 'client_id':'# clients'})
#calculate the renewal rate per industry
ind_ren_nums['renewal_rate'] = np.round(ind_ren_nums['# renewed']/ind_ren_nums['# clients'], 3)
display(ind_ren_nums)

#obtain the industry with the highest renewal rate
top_industry = ind_ren_nums.nlargest(1,'renewal_rate')['industry'].values[0]
#the total # clients
top_ind_tot = ind_ren_nums[ind_ren_nums.industry == top_industry]['# clients'].values[0]
#the # clients who renewed
top_ind_ren = ind_ren_nums[ind_ren_nums.industry == top_industry]['# renewed'].values[0]

display(f"The {top_industry} industry saw the highest subscription renewal rate among their clients. Of their {top_ind_tot} total clients, {top_ind_ren} of them renewed their subscription.")

,industry,# renewed,# clients,renewal_rate
0,AI,7,11,0.636
1,Crypto,11,25,0.440
2,E-commerce,9,20,0.450
3,Fintech,12,22,0.545
4,Gaming,16,22,0.727


'The Gaming industry saw the highest subscription renewal rate among their clients. Of their 22 total clients, 16 of them renewed their subscription.'

# Analysis III
**For clients that renewed their subscriptions, what was the average inflation rate when their subscriptions were renewed?**

It is assumed that subscriptions are renewed when they expire/end which is indicated via the 'end_date' variable from the subscriptions data.  
To merge the subscriptions & economic-indicator datasets, the date variables must be used; however, the dates don't line up exactly because certain dates (from the subscription data) are being matched to ranges of dates (from the economic data). To execute this, `pandas.merge_asof` can be used to look for the most recent 'start_date' in the economic data based on the 'end_date' (renewal date) in the subscriptions data.
- Note that with `pandas.merge_asof`, the variables being used to join the data need to be sorted. Fortunately, the economic data is sorted to begin with.

In [57]:
#filter for clients that renewed their subscription, sort the data according to the subscription-renewal date ('end_date')
ren_df = subscrp_df[subscrp_df.renewed == True].sort_values('end_date')

#merge the data
renewals_eco_df = pd.merge_asof(ren_df, eco_df, left_on='end_date', right_on='start_date', suffixes=("_subscrp", "_eco"), 
                                direction='backward')

#calculate the avg inflation rate
average_inflation_for_renewals = renewals_eco_df['inflation_rate'].mean()
display(f"For clients that renewed their subscriptions, the average inflation rate when their subscriptions were renewed is about {np.round(average_inflation_for_renewals,2)}.")

'For clients that renewed their subscriptions, the average inflation rate when their subscriptions were renewed is about 4.42.'